# Q.16 연구소

##### 크기가 N x M인 연구소는 빈칸(0), 벽(1), 바이러스(2)으로 이루어져 있는데 벽 3개를 새로 세워 얻을 수 있는 안전 영역(바이러스가 없는, 벽으로 둘러싸인 공간의 합) 크기의 최대값을 구해라
##### https://www.acmicpc.net/problem/14502

##### <답안1>
#####

In [ ]:
from itertools import combinations
from collections import deque
from copy import deepcopy

# 지도의 크기
n, m = map(int, input().split())

# 연구실 정보 저장
lab = []
for _ in range(n):
    lab.append(list(map(int, input().split())))

# 빈 공간의 좌표를 모두 저장
empty = []
for i in range(n):
    for j in range(m):
        if lab[i][j] == 0:
            empty.append((i,j))

# 가능한 울타리 조합을 모두 계산
total_empty_combi = list(combinations(empty, 3))

# 상하좌우 이동용 좌표
dx = [-1, 1, 0, 0]
dy = [0, 0, -1, 1]

# BFS를 사용해 바이러스 확산시킨 뒤, 남은 안전 영역을 계산
def bfs():

    # 울타리가 설치된 지도를 깊은 복사
    lab_copy = deepcopy(lab)

    for x in range(n):
        for y in range(m):

            # 바이러스 확산용 큐
            q = deque()

            # 현재 위치에 바이러스가 있다면
            if (lab_copy[x][y] == 2):

                # 현재 위치를 큐에 추가
                q.append((x, y))
                while(q):

                    # 현재 위치를 큐에서 추출
                    now = q.popleft()

                    # 상하좌우의 빈 공간에 바이러스를 학산
                    for i in range(4):
                        new_x = now[0] + dx[i]
                        new_y = now[1] + dy[i]

                        # 이웃 영역이 지도 내에 존재하고, 빈 공간인지 확인
                        if (0 <= new_x and new_x < n and 0 <= new_y and new_y < m and lab_copy[new_x][new_y] == 0):
                            # 주변 위치에 바이러스를 확산
                            lab_copy[new_x][new_y] = 2

                            # 이웃 공간을 큐에 추가
                            q.append((new_x, new_y))

    return safe_area(lab_copy)

# 남은 안전 영역을 계산
def safe_area(lab_copy):

    count = 0
    for i in range(n):
        for j in range(m):
            if (lab_copy[i][j] == 0):
                count += 1

    return count

def solution():

    # 울타리 설치 전 원본
    global lab

    # 울타리 조합 별 안전 영역의 개수 저장
    result = -1

    # 울타리를 세운 뒤, BFS를 사용해 바이러스를 확산시키고, 남은 안전 영역을 계산
    for empty_combi in total_empty_combi:

        # 울타리 설치
        for x, y in empty_combi:
            lab[x][y] = 1

        # BFS를 사용해 바이러스 확산시킨 뒤, 남은 안전 영역을 계산하고, 최대값과 비교
        result = max(result, bfs())

        # 울타리 제거
        for x, y in empty_combi:
            lab[x][y] = 0

    return result

print(solution())

##### 답안2(시간초과)

In [ ]:
n, m = map(int, input().split())
temp = [[0] * m for _ in range(n)] # 벽을 설치한 뒤의 맵 리스트
graph = [list(map(int, input().split())) for _ in range(n)] # 초기 맵 리스트

result = 0

# 4가지 이동 방향에 대한 리스트
dx = [-1, 0, 1, 0]
dy = [0, 1, 0, -1]

# 깊이 우선 탐색을 이용해 각 바이러스가 사방으로 퍼지도록 하기
def virus(x, y):
    for i in range(4):
        nx = x + dx[i]
        ny = y + dy[i]
        # 상,하,좌,우 중에서 바이러스가 퍼질 수 있는 경우
        if nx >= 0 and nx < n and ny >= 0 and ny < m:
            if temp[nx][ny] == 0:
                # 해당 위치에 바이러스를 배치하고, 다시 재귀적으로 수행
                temp[nx][ny] = 2
                virus(nx, ny)

# 현재 맵에서 안전 영역의 크기 계산하는 메서드
def get_score():
    score = 0
    for i in range(n):
        for j in range(m):
            if temp[i][j] == 0:
                score += 1
    return score

# 깊이 우선 탐색을 이용해 울타리를 설치하면서, 매번 안전 영역의 크기 계산
def dfs(count):
    global result
    # 울타리가 3개 설치된 경우
    if count == 3:
        for i in range(n):
            for j in range(m):
                temp[i][j] = graph[i][j]
        # 각 바이러스의 위치에서 전파 진행
        for i in range(n):
            for j in range(m):
                if temp[i][j] == 2:
                    virus(i, j)
        # 안전 영역의 최댓값 계산
        result = max(result, get_score())
        return
    # 빈 공간에 울타리 설치
    for i in range(n):
        for j in range(m):
            if graph[i][j] == 0:
                graph[i][j] = 1
                count += 1
                dfs(count)
                graph[i][j] = 0
                count -= 1

dfs(0)
print(result)